In [1]:
from dotenv import load_dotenv; load_dotenv()
import os
import json
from datetime import datetime, timezone, timedelta
import pandas as pd
import requests
import sys; sys.path.append("./Backend")

In [2]:
from Backend.Order import Order
from Backend.Account import Account
from Backend.HistoryData import getHistoryBarsData
from Backend.Resolution import Resolution
from Indicator.SMA import SMA
from Backend.Algorithm import Algorithm
from Backend.config import USATimeZone
from Backend.RollingWindow import RollingWindow
from TestAlgo import TestAlgo

In [25]:
algo = TestAlgo(log=True, debug_log=True)
algo._set_backtest()
algo.Initialize()
history = algo._get_backtest_history_data_in()

[History data]
start: 2021-07-28 15:39:00-04:00
end: 2021-07-28 15:59:00-04:00
[History data]
start: 2021-07-28 15:39:00-04:00
end: 2021-07-28 15:59:00-04:00
[History data]
start: 2021-07-29 00:00:00-04:00
end: 2021-07-30 00:00:00-04:00[History data]
start: 2021-07-29 00:00:00-04:00
end: 2021-07-30 00:00:00-04:00



In [26]:
history

open    high     low   close  volume
symbol timestamp                                                        
SPY    2021-07-29 08:00:00+00:00  438.93  439.27  438.93  439.24     407
       2021-07-29 08:02:00+00:00  439.30  439.30  439.30  439.30     225
       2021-07-29 08:07:00+00:00  439.14  439.14  439.14  439.14     247
       2021-07-29 08:08:00+00:00  439.22  439.26  439.22  439.24    1439
       2021-07-29 08:16:00+00:00  439.46  439.46  439.46  439.46     100
...                                  ...     ...     ...     ...     ...
TSLA   2021-07-29 23:55:00+00:00  668.18  668.18  668.10  668.10    1026
       2021-07-29 23:56:00+00:00  668.50  668.50  668.50  668.50    1103
       2021-07-29 23:57:00+00:00  668.90  668.96  668.50  668.96    1265
       2021-07-29 23:58:00+00:00  668.72  668.74  668.72  668.74    1272
       2021-07-29 23:59:00+00:00  668.99  669.00  668.99  668.99    4082

[1498 rows x 5 columns]

In [13]:
current = algo.Time.now
index = pd.Timestamp(current.astimezone(timezone.utc))
data = history.loc[(slice(None), slice(index)), :]

In [33]:
now = pd.Timestamp(datetime(2021,7,29,8,tzinfo=timezone.utc))
end = pd.Timestamp(datetime(2021,7,30,tzinfo=timezone.utc))
cnt = 0
while now < end:
    try:
        history.reset_index().set_index('timestamp')
        now += timedelta(minutes=1)
        cnt += 1
    except KeyError:
        pass

,symbol,open,high,low,close,volume
timestamp,,,,,,
2021-07-29 08:00:00+00:00,SPY,438.93,439.27,438.93,439.24,407
2021-07-29 08:02:00+00:00,SPY,439.30,439.30,439.30,439.30,225
2021-07-29 08:07:00+00:00,SPY,439.14,439.14,439.14,439.14,247
2021-07-29 08:08:00+00:00,SPY,439.22,439.26,439.22,439.24,1439
2021-07-29 08:16:00+00:00,SPY,439.46,439.46,439.46,439.46,100
...,...,...,...,...,...,...
2021-07-29 23:55:00+00:00,TSLA,668.18,668.18,668.10,668.10,1026
2021-07-29 23:56:00+00:00,TSLA,668.50,668.50,668.50,668.50,1103
2021-07-29 23:57:00+00:00,TSLA,668.90,668.96,668.50,668.96,1265


In [10]:
for i in history[-10:].itertuples():
    print(i)

Pandas(Index=('TSLA', Timestamp('2021-07-29 23:49:00+0000', tz='UTC')), open=668.61, high=669.4, low=668.41, close=669.4, volume=1221)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:50:00+0000', tz='UTC')), open=669.04, high=669.04, low=668.5, close=669.0, volume=1318)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:51:00+0000', tz='UTC')), open=668.95, high=668.95, low=668.95, close=668.95, volume=669)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:52:00+0000', tz='UTC')), open=668.36, high=668.37, low=668.0, close=668.0, volume=3312)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:53:00+0000', tz='UTC')), open=668.01, high=668.47, low=668.0, close=668.0, volume=1345)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:55:00+0000', tz='UTC')), open=668.18, high=668.18, low=668.1, close=668.1, volume=1026)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:56:00+0000', tz='UTC')), open=668.5, high=668.5, low=668.5, close=668.5, volume=1103)
Pandas(Index=('TSLA', Timestamp('2021-07-29 23:57:00+000

In [74]:
# df.loc[:,pd.Timestamp('2021-07-30 18:06:00+00:00')]
df.loc['SPY'].loc[day]
# df.loc[(slice(None), slice(day)), :]

open         438.5699
high         438.6000
low          438.4400
close        438.4599
volume    145433.0000
Name: 2021-07-30 19:21:00+00:00, dtype: float64

In [70]:
df

open      high       low     close  \
symbol timestamp                                                           
TSLA   2021-07-30 19:21:00+00:00  684.6021  684.9600  684.2800  684.3300   
       2021-07-30 19:22:00+00:00  684.4700  684.6899  683.5500  683.7571   
       2021-07-30 19:23:00+00:00  683.7700  684.1713  683.3023  683.7599   
       2021-07-30 19:24:00+00:00  683.7188  683.8500  683.0000  683.1000   
       2021-07-30 19:25:00+00:00  683.2050  684.4500  683.1950  684.2800   
       2021-07-30 19:26:00+00:00  684.2600  684.7800  684.0000  684.7550   
       2021-07-30 19:27:00+00:00  684.6300  684.7888  683.7800  683.8900   
       2021-07-30 19:28:00+00:00  684.0200  684.5900  684.0120  684.2700   
       2021-07-30 19:29:00+00:00  684.2300  685.0899  683.9500  684.9000   
       2021-07-30 19:30:00+00:00  684.7600  685.1800  684.6101  685.1274   
SPY    2021-07-30 19:21:00+00:00  438.5699  438.6000  438.4400  438.4599   
       2021-07-30 19:22:00+00:00  438.4550  438.5200  438.3000  438.3300   
       2021-07-30 19:23:00+00:00  438.3399  438.3399  438.2050  438.2300   
       2021-07-30 19:24:00+00:00  438.2250  438.2250  438.0900  438.1299   
       2021-07-30 19:25:00+00:00  438.1200  438.2900  438.1000  438.2799   
       2021-07-30 19:26:00+00:00  438.2800  438.4100  438.2800  438.3700   
       2021-07-30 19:27:00+00:00  438.3600  438.4700  438.3400  438.4599   
       2021-07-30 19:28:00+00:00  438.4500  438.5200  438.4100  438.4100   
       2021-07-30 19:29:00+00:00  438.4100  438.4700  438.3601  438.4299   
       2021-07-30 19:30:00+00:00  438.4300  438.6000  438.4200  438.6000   

                                  volume  
symbol timestamp                          
TSLA   2021-07-30 19:21:00+00:00   37296  
       2021-07-30 19:22:00+00:00   25175  
       2021-07-30 19:23:00+00:00   24791  
       2021-07-30 19:24:00+00:00   59479  
       2021-07-30 19:25:00+00:00   41182  
       2021-07-30 19:26:00+00:00   33089  
       2021-07-30 19:27:00+00:00   30762  
       2021-07-30 19:28:00+00:00   16421  
       2021-07-30 19:29:00+00:00   39816  
       2021-07-30 19:30:00+00:00   42842  
SPY    2021-07-30 19:21:00+00:00  145433  
       2021-07-30 19:22:00+00:00  116086  
       2021-07-30 19:23:00+00:00  160353  
       2021-07-30 19:24:00+00:00  126652  
       2021-07-30 19:25:00+00:00  118956  
       2021-07-30 19:26:00+00:00  136716  
       2021-07-30 19:27:00+00:00  175332  
       2021-07-30 19:28:00+00:00  105932  
       2021-07-30 19:29:00+00:00  166864  
       2021-07-30 19:30:00+00:00  130388

In [9]:
algo.Account._getAccountInfo()

{'id': 'e7d13ae8-488a-49c1-b9f6-cc30a8e17c43',
 'account_number': 'PA32JJXCO8U5',
 'status': 'ACTIVE',
 'currency': 'USD',
 'buying_power': '409346.16',
 'regt_buying_power': '204673.08',
 'daytrading_buying_power': '409346.16',
 'cash': '102336.54',
 'portfolio_value': '102336.54',
 'pattern_day_trader': True,
 'trading_blocked': False,
 'transfers_blocked': False,
 'account_blocked': False,
 'created_at': '2021-07-28T12:43:40.987144Z',
 'trade_suspended_by_user': False,
 'multiplier': '4',
 'shorting_enabled': True,
 'equity': '102336.54',
 'last_equity': '102336.54',
 'long_market_value': '0',
 'short_market_value': '0',
 'initial_margin': '0',
 'maintenance_margin': '0',
 'last_maintenance_margin': '0',
 'sma': '102336.54',
 'daytrade_count': 27}

In [23]:
a['symbol'] = 'TSLA'
b['symbol'] = 'GOOGL'
a = pd.concat([a,b])
a.reset_index().set_index(['symbol', 'timestape'])

open       high        low      close  \
symbol timestape                                                               
TSLA   2021-07-29 19:50:00+00:00   675.1000   675.8500   675.0800   675.2497   
       2021-07-29 19:51:00+00:00   675.3037   675.4300   674.3400   674.5400   
       2021-07-29 19:52:00+00:00   674.5100   675.1700   674.4000   674.9801   
       2021-07-29 19:53:00+00:00   675.0000   675.2500   674.7400   675.1500   
       2021-07-29 19:54:00+00:00   675.1100   675.4800   674.8410   675.1200   
       2021-07-29 19:55:00+00:00   675.2500   677.0000   675.1200   677.0000   
       2021-07-29 19:56:00+00:00   677.0000   678.0000   676.9700   677.8100   
       2021-07-29 19:57:00+00:00   677.8200   678.1100   676.8000   677.1200   
       2021-07-29 19:58:00+00:00   677.1600   677.2869   676.4600   677.1701   
       2021-07-29 19:59:00+00:00   677.2000   677.7400   677.0401   677.3600   
GOOGL  2021-07-29 19:50:00+00:00  2712.0000  2712.5800  2710.8600  2710.8600   
       2021-07-29 19:51:00+00:00  2711.2900  2712.9500  2711.1900  2712.9500   
       2021-07-29 19:52:00+00:00  2712.3450  2713.1200  2711.9000  2713.1200   
       2021-07-29 19:53:00+00:00  2714.2900  2715.2799  2713.6400  2715.2799   
       2021-07-29 19:54:00+00:00  2714.2400  2716.0900  2713.8600  2715.5600   
       2021-07-29 19:55:00+00:00  2714.9400  2716.3700  2714.3500  2715.8800   
       2021-07-29 19:56:00+00:00  2715.8505  2717.1450  2715.8505  2716.4700   
       2021-07-29 19:57:00+00:00  2716.1600  2717.3800  2715.3500  2716.1000   
       2021-07-29 19:58:00+00:00  2715.9100  2717.4100  2715.0675  2716.5300   
       2021-07-29 19:59:00+00:00  2717.0300  2717.0800  2715.5500  2715.9134   

                                  volume  
symbol timestape                          
TSLA   2021-07-29 19:50:00+00:00  116058  
       2021-07-29 19:51:00+00:00  104945  
       2021-07-29 19:52:00+00:00   85902  
       2021-07-29 19:53:00+00:00   66490  
       2021-07-29 19:54:00+00:00   72394  
       2021-07-29 19:55:00+00:00  118640  
       2021-07-29 19:56:00+00:00  234331  
       2021-07-29 19:57:00+00:00  192510  
       2021-07-29 19:58:00+00:00  143167  
       2021-07-29 19:59:00+00:00  179490  
GOOGL  2021-07-29 19:50:00+00:00    7960  
       2021-07-29 19:51:00+00:00    6045  
       2021-07-29 19:52:00+00:00    6310  
       2021-07-29 19:53:00+00:00    4522  
       2021-07-29 19:54:00+00:00   11442  
       2021-07-29 19:55:00+00:00    9887  
       2021-07-29 19:56:00+00:00    6733  
       2021-07-29 19:57:00+00:00   16355  
       2021-07-29 19:58:00+00:00   16141  
       2021-07-29 19:59:00+00:00   15648